## Notebook orcid-base
Fetch information about a specific person identified by his/her ORCID ID and list all co-authors.

In [186]:
import sys
sys.path.append('../')

# !{sys.executable} -m pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import altair as alt
import networkx as nx
import nx_altair as nxa
import pandas as pd
from datetime import datetime, date
import requests
import json

import os
from os.path import join, dirname
from dotenv import load_dotenv
load_dotenv()

True

In [187]:
# ORCID ID does not exist
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0003-XXXX-XXXX"

url = f'https://augment.azure-api.net/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 200 and not r.json()[0]["nodes"]["researchers"]:
    print(f'Researcher with ORCID ID {ORCID} not found.')

Augment API query complete  200
Researcher with ORCID ID 0000-0003-XXXX-XXXX not found.


In [188]:
# ORCID ID does exist
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0003-1419-2405"

url = f'https://augment.azure-api.net/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 200 and r.json()[0]["nodes"]["researchers"]:    
    researchers = r.json()[0]["nodes"]["researchers"]
    
    researcher = None
    for i in range(len(researchers)):
        if researchers[i]["orcid"] == ORCID:
            researcher = researchers[i]
            
if r.status_code == 200 and r.json()[0]["nodes"]["publications"]:    
    publications = r.json()[0]["nodes"]["publications"]
    
if r.status_code == 200 and r.json()[0]["nodes"]["organisations"]:    
    organisations = r.json()[0]["nodes"]["organisations"]

Augment API query complete  200


In [189]:
print(f'The researcher {researcher["full_name"]} is connected to {len(researchers)} researchers.')
print(f'The researcher {researcher["full_name"]} is connected to {len(publications)} publications.')
print(f'The researcher {researcher["full_name"]} is connected to {len(organisations)} organisations.')

The researcher Martin Fenner is connected to 129 researchers.
The researcher Martin Fenner is connected to 126 publications.
The researcher Martin Fenner is connected to 298 organisations.


In [190]:
pd.set_option("display.max_rows", None)
researchers_df = pd.DataFrame(researchers, columns=['orcid', 'full_name'])
display(researchers_df)

publications_df = pd.DataFrame(publications, columns=['doi', 'publication_year', 'title'])
display(publications_df)

organisations_df = pd.DataFrame(organisations, columns=['country', 'name'])
display(organisations_df)

countries_df = organisations_df.groupby('country').nunique()
display(countries_df)
labels = countries_df[countries_df.columns[1:]].to_numpy()
display(labels)

,orcid,full_name
0,0000-0001-5492-3212,Markus Stocker
1,0000-0002-9576-1974,Stephanie van de Sandt
2,0000-0001-5801-1886,Zohreh Zahedi
3,0000-0002-8689-4935,Josh Brown
4,0000-0002-1611-6935,Jo McEntyre
5,0000-0002-6513-4996,David Vieglais
6,0000-0001-9623-2225,Catriona MacCallum
7,0000-0002-5193-7851,Andrea Mannocci
8,0000-0001-6843-7960,Rachael Kotarski
9,0000-0001-5109-3700,Laurel Haak


,doi,publication_year,title
0,10.5281/zenodo.3340591,2018.0,Code of practice for research data usage. Metr...
1,10.5281/zenodo.3606060,2020.0,Deliverable 4.4 Organizational IDs in Practice
2,10.1093/annonc/mds579,2012,"Maintaining success, reducing treatment burden..."
3,10.1002/bult.2013.1720390409,2013,The many faces of article-level metrics
4,10.5281/zenodo.3248652,2019.0,D2.2 PID Metadata Provenance
5,10.1007/978-3-030-11226-4_7,2019,Connecting Researchers to Data Repositories in...
6,10.1007/s00345-010-0527-5,2010,Carboplatin plus weekly docetaxel as salvage c...
7,10.5281/zenodo.1324300,2018.0,D2.1 Pid Resolution Services Best Practices
8,10.5281/zenodo.1324296,2018.0,D3.1 Survey Of Current Pid Services Landscape
9,10.5438/0007-nw90,2015.0,DataCite Blog


,country,name
0,NL,Universitair Medisch Centrum Groningen
1,DE,Freie Universität Berlin Fachbereich Geowissen...
2,US,University of Michigan
3,US,Cornell University
4,CN,National Center for Protein Science Beijing
5,US,The Chan Zuckerberg Initiative
6,DE,National Center for Biotechnology Research
7,CH,Oncology Institute of Southern Switzerland (IOSI)
8,US,University of California Curation Center (UC3)
9,ES,Universidad Politécnica de Madrid


country  name                                                                                                  
AF       common:name                                                                                               1
US       Cell Genesys                                                                                              1
         Cedars-Sinai                                                                                              1
         California Polytechnic State University                                                                   1
         California Digital Library                                                                                1
         CHORUS                                                                                                    1
         Brown University                                                                                          1
         Boston University                                           

AttributeError: 'Series' object has no attribute 'columns'